In [2]:
!pip3 install setuptools wheel
!pip3 install spacy
!python3 -m spacy download en_core_web_sm
!pip3 install nltk
import spacy
import nltk
from nltk.tokenize import word_tokenize
nltk.download('omw-1.4')
from nltk.corpus import wordnet
import re
import csv

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
     |████████████████████████████████| 13.9 MB 2.7 MB/s eta 0:00:01    |███████▉                        | 3.4 MB 1.2 MB/s eta 0:00:09


You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


[nltk_data] Downloading package omw-1.4 to /Users/Varvara/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
nlp = spacy.load("en_core_web_sm")

**Поиск лексической избыточности в сочинительных конструкциях.**

In [4]:
def get_synonym(phrase):
    synonyms = []

    for syn in wordnet.synsets(phrase):
        for l in syn.lemmas():
            if "_" in l.name():
                synonyms.append(" ".join(l.name().split("_"))) # замена нижних подчеркиваний на пробелы
            else:
                synonyms.append(l.name())    
    return list(set(synonyms))

In [5]:
def get_syn_red(sent):
    syn_spacy = nlp(sent)
    syn_red = []
    for token in syn_spacy:
        if token.dep_ == 'conj':
            if token.lemma_ == token.head.lemma_:
                syn_red.append(token.head.text)
                syn_red.append(token.text)               
            else:
                for syn in get_synonym(token.text):
                    if syn == token.head.lemma_:
                        syn_red.append(token.head.text)
                        syn_red.append(token.text)
    return syn_red

**Поиск дву вершин – предлогов в предложной группе.**

In [6]:
def get_adp_red(sent):
    prep_spacy = nlp(sent)
    prob_red = list()
    for i in range(len(prep_spacy)):
        if prep_spacy[i].pos_ == 'ADP' and prep_spacy[i + 1].pos_ == 'ADP':
            red = prep_spacy[i].text + ' ' + prep_spacy[i + 1].text
            prob_red.append(red)
    return prob_red

**Предобработка для применения функций и записи в файл.**

In [7]:
def get_dict(name, sent, func):
    red_dict = {}
    red_dict['text_name'] = name
    red_dict['error_sentence'] = sent
    red_dict['probable_redundancy'] = func
    return red_dict 

In [8]:
with open('YYYY-D-D-TTT-S.txt', encoding='utf-8') as file:
    file_r = file.read()
    name = 'YYYY-D-D-TTT-S.txt'
    text = file_r.split('\n')# деление по репликам
    list_sentences = []
    for replicа in text:
        filt_replica = re.sub(r'<\/?\w>', r'', replicа)# удаление тегов
        sentences = filt_replica.split('.')
        for i in sentences:
            if i != '':
                new_i = i + '.'
                list_sentences.append(new_i)
    final_sentences = list()
    for sent in list_sentences:
        filt_1_sent = re.sub(r'\(\w*\) ', r'', sent)# удаление по типу (um), (erm), (er)
        filt_2_sent = re.sub(r'\w*= ', r'', filt_1_sent)# удаление запинок и оговорок по типу 
        filt_3_sent = re.sub(r'^ ', r'', filt_2_sent)# удаление лишних пробелов в начале
        final_sentences.append(filt_3_sent)
#print(final_sentences) 

In [9]:
list_dict = []
for s in final_sentences:
    if len(get_adp_red(s)) != 0 :
        list_dict.append(get_dict(name, s, get_adp_red(s)))       
    elif len(get_syn_red(s)) != 0 :
        list_dict.append(get_dict(name, s, get_syn_red(s)))        
#print(list_dict)

In [10]:
# запись обнаруженных избыточностей в таблицу 
with open('redundancy.csv', 'w', newline='') as csvfile:
    fieldnames = ['text_name', 'error_sentence', 'probable_redundancy']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in list_dict:
        writer.writerow(row)